In [ ]:
import basic
import geopandas as gpd
import pandas as pd
import numpy as np
import flopy
import pathlib
import os
import pyemu
import Hydrographs
import matplotlib.pyplot as plt
import conda_scripts.arich_functions as af
import conda_scripts.rich_gis as rg
import pest_post_process
%matplotlib inline

In [ ]:
run = 'ibound_update'
info, swr_info, sfr_info, riv_keys_info = basic.load_params(run)

datestart = info['start_date']
numdays = info['numdays']
name = info['name']

out_folder = basic.out_folder(run)


print(datestart)
print(out_folder)


In [ ]:
ml = basic.load_model()

In [ ]:
import write_inflows
import write_pond_inflows_rch
import make_wells
import initial_conditions

In [ ]:
# import matplotlib.dates as mdates
# fig, ax = plt.subplots(figsize=(9, 6))


# bottom = df.copy()
# bottom.insert(0, 0, 0)
# # modify the base of each columns, can do with a for loop
# for col in df.columns:

#     bot = bottom.cumsum(axis=1).shift(axis=1).loc[:,col]
        
#     ax.bar(df.droplevel(1,0).index, df.droplevel(1,0).cumsum(axis = 1).loc[:, col].values, bottom = bot, label = col) 

# ax.grid(True)
# ax.set(ylabel='acre-feet', title="Total Caisson Pumping, per Well")
# basic.set_dates_xtick(ax)
    
# ax.legend()

In [ ]:
basic.setup_folder(run)
write_inflows.run(model_name=run, m = ml, numdays=numdays, datestart=datestart)
df = make_wells.run(name=run, m = ml, datestart=datestart, numdays=numdays)
write_pond_inflows_rch.run(run, m = ml, draw_maps = True, add_overland = True, ovr_flux = 0.5)

# shutil.rmtree(os.path.join(new_folder, 'Results'))
# os.mkdir(os.path.join(new_folder, 'Results'))
# new_folder = os.path.join()
# src = os.path.join('initial_heads', f'June2015')
# print(f'copying initial heads from {src} to {ml.model_ws}')
# initial_conditions.set_start_from_path(src, ml.model_ws)

In [ ]:
# def get_linear_analysis(ml):
    
#     return LA = pyemu.LinearAnalysis(jco=os.path.join(ml.model_ws,'reg_pest.jco'),
#                  pst=os.path.join(ml.model_ws,'reg_pest.pst') )

# def par_sensitivity(LA, out_folder):
    
#     '''
#     get parameter and observation sensitivity
#     '''

#     sens = LA.get_par_css_dataframe()
#     sens.loc[:,['pest_css']].sort_values('pest_css').plot.barh(title = 'Parameter Sensitivities', figsize = (9,6))
#     plt.savefig(os.path.join(out_folder, 'parsen.png'), bbox_inches = 'tight')
    
# def obs_sensitivity(LA, out_folder):
#     ccs = LA.get_cso_dataframe()
    
#     ccs.loc[:,'type'] = ccs.index.map(lambda x: 'drawdown' if x.startswith('d') else 'obs')

#     ccs.loc[:,'well'] = ccs.index.map(lambda x: 'LRR{0:04s}'.format( x.replace('d','').replace('l', '').split('.')[0]))
    
#     ccs.groupby(['well', 'type']).mean().unstack(1).droplevel(0,1)\
#     .sort_values('obs', ascending = False).rename(columns = {'obs':'head'})\
#     .plot.bar(title ="Mean Observation Sensitivity", xlabel = '', figsize = (9,6))
#     plt.savefig(os.path.join(out_folder, 'obssen.png'), bbox_inches = 'tight')
    
#     return css

# def map_css_mean(ccs):
#     wells_mod = gpd.read_file('GIS/wells_mod.geojson')
#     wells_mod = wells_mod.set_index('station_name').join(ccs.groupby(['well', 'type']).mean(
#     ).unstack(1).droplevel(0,1))

#     wells_mod.loc[:,'obslabel'] = wells_mod.loc[:,'obs'].apply(lambda x: f"{x:.2f}")
#     wells_mod.loc[:,'drawdownlabel'] = wells_mod.loc[:,'drawdown'].apply(lambda x: f"{x:.2f}")

#     for col in ['obs', 'drawdown']:
#         fig, ax = basic.basic_map(m = ml)

#         if col == 'obs':
#             fac = 50
#         else:
#             fac = 200

#         cdf = wells_mod.dropna(subset = col)
#         cdf.plot(col, cmap = 'bwr',
#                        legend = True,
#                         legend_kwds={
#                                     "location":"right",
#                                     "shrink":.2
#                                     },
#                        ax=  ax, 
#                        markersize  = cdf.loc[:,col]*50, 
#                        edgecolor = 'k')

#         basic.set_bounds(ax)
#         v = col.replace('obs','Observed Head')
#         ax.set_title(f"Mean {v} Sensitivities")
#         ax.text(0,1, 'Average Sensitivities Per Well', va ='top',transform = ax.transAxes)
#         plt.draw()
#         rg.label_points(ax, cdf, basin_name=None, colname = col+'label', already_str = True)
#         plt.savefig(os.path.join(out_folder, f'obssen_map_{col}.png'), bbox_inches = 'tight')
        
# def map_css_total(ccs):
#     wells_mod = gpd.read_file('GIS/wells_mod.geojson')
#     wells_mod = wells_mod.set_index('station_name').join(ccs.groupby(['well', 'type']).sum(
#     ).unstack(1).droplevel(0,1))

#     wells_mod.loc[:,'obslabel'] = wells_mod.loc[:,'obs'].apply(lambda x: f"{x:.0f}")
#     wells_mod.loc[:,'drawdownlabel'] = wells_mod.loc[:,'drawdown'].apply(lambda x: f"{x:.0f}")

#     for col in ['obs', 'drawdown']:
#         fig, ax = basic.basic_map(m = ml)
#         if col == 'obs':
#             fac = .1
#         else:
#             fac = .3
#         cdf = wells_mod.dropna(subset = col)
#         cdf.plot(col, cmap = 'bwr',
#                             legend = True,
#                             legend_kwds={
#                             "location":"right",
#                             "shrink":.2
#                             },
#                        ax=  ax, 
#                        markersize  = cdf.loc[:,col]*fac, 
#                        edgecolor = 'k')

#         basic.set_bounds(ax)
#         v = col.replace('obs','Observed Head')
#         ax.set_title(f"Total {v} Sensitivities")
#         ax.text(0,1, 'Total Sensitivities Per Well', va ='top',transform = ax.transAxes)
#         plt.draw()
#         rg.label_points(ax, cdf, basin_name=None, colname = col+'label', already_str = True)
#         plt.savefig(os.path.join(out_folder, f'obssen_map_{col}_Total.png'), bbox_inches = 'tight')

In [ ]:
sens.sort_values(by = 'pest_css', ascending = True).tail(40).plot.barh(figsize = (4,10))

In [ ]:
LA = pest_post_process.get_linear_analysis(ml,'pp_pest')
sens = pest_post_process.par_sensitivity(LA, out_folder)
ccs = pest_post_process.obs_sensitivity(LA, out_folder=out_folder)
pest_post_process.map_css_mean(ccs, ml, out_folder)
pest_post_process.map_css_total(ccs, ml, out_folder)

In [ ]:
basic.setup_folder(run)

In [ ]:
Hydrographs.run(run, reload = True, ml = ml, add_temp = False)

In [ ]:
rc_df = pd.read_csv(os.path.join("versions",run,'hydrographs', "all_meas.csv"), index_col = [0], parse_dates=True)
rc_df.loc[:,'well_num'] = rc_df.loc[:,'well'].apply(lambda x: 'l'+str(x.split('LRR')[1]))
rc_df.loc[:,'well_num_dd'] = rc_df.loc[:,'well'].apply(lambda x: 'd'+str(x.split('LRR')[1]))
rc_df.loc[:,'kper'] = (rc_df.index - pd.to_datetime(datestart)).days
rc_df = rc_df.dropna(subset = 'Observed')
# hds = flopy.utils.HeadFile(os.path.join(ml.model_ws,"freyberg.hds"))
# data = hds.get_alldata()
obs = []
roff = 0.0#ml.dis.delc.array[0] / 2.0
coff = 0.0#ml.dis.delr.array[0] / 2.0
# for n,r,c in zip(rc_df.name,rc_df.row,rc_df.col):
# for kper in range(numdays):
for well, r in rc_df.groupby('well_num'):
    d = np.zeros((r.shape[0],2))
    d[:,0] = r.kper.values*86400
    d[:,1] = r.Observed.values

    # names =     ["{:}.{:}".format(r.well.unique()[0],x) for x in r.kper.values]
    names =     ["{:}.{:}".format(well,x) for x in r.kper.values]
   
    obs.append(flopy.modflow.HeadObservation(ml,
                obsname=well,
                layer=0,row=r.loc[:,'i_r'].unique()[0],
                column=r.loc[:,'j_c'].unique()[0],roff=roff,coff=coff,
                time_series_data=d,
                names= names))
    
# add drawdown 
for well, r in rc_df.groupby('well_num_dd'):
    
    #get every 5 measurements
    for irow in range(0,r.shape[0], 5):
        cdf = r.iloc[irow:irow+5]
        d = np.zeros((cdf.shape[0],2))
        d[:,0] = cdf.kper.values*86400
        d[:,1] = cdf.Observed.values

        names = ["{:}.{:}".format(well,x) for x in cdf.kper.values]

        obs.append(flopy.modflow.HeadObservation(ml,
                    irefsp = -1,       
                    obsname=well, itt = 2,
                    layer=0,row=cdf.loc[:,'i_r'].unique()[0],
                    column=cdf.loc[:,'j_c'].unique()[0],
                    roff=roff,coff=coff,
                    time_series_data=d,
                    names= names))
    
hobs = flopy.modflow.ModflowHob(ml,obs_data=obs,iuhobsv=600)
hobs.write_file()
# ext_path = os.path.join(ml.model_ws,"ref")
# if os.path.exists(ext_path):
#     shutil.rmtree(ext_path)
# print(ext_path)
# os.mkdir(ext_path)
# ml.external_path = os.path.split(ext_path)[-1]
# ml.upw.hk.fmtin = "(FREE)"
# ml.upw.sy.fmtin = "(FREE)"
# ml.rch.rech.fmtin = "(FREE)"
# ml.write_input()
# ml.run_model()

In [ ]:
# def drawdown_hobs(folder, filename = r".\rr_tr.hob"):

#     """
#     from ayman - still not implemented
#     A script to read hob.out file and compute simulated and observed drawdown.

#     """

#     # read hob file
    
#     hob_file = os.path.join(folder, filename)

#     df = pd.read_csv(hob_file, delim_whitespace=True)

#     site_info = df['OBSERVATION NAME'].str.split(".", expand=True)
#     df['site_id'] = site_info[0]
#     df['time_index'] = site_info[1]
#     df.loc[df['time_index'].isna(), 'time_index'] = 0


#     def compute_draw_down(_df):
#         if len(_df) == 1:
#             _df['sim_drawdown'] = 0
#             _df['obs_drawdown'] = 0
#             return _df
#         else:
#             # sort heads by time
#             _df = _df.sort_values(by='time_index')

#             _df['sim_drawdown'] = _df['SIMULATED EQUIVALENT'].diff()
#             _df['obs_drawdown'] = _df['OBSERVED VALUE'].diff()

#             # first head has no drawdown-- make it zero instead of nan
#             _df.loc[_df['sim_drawdown'].isna(), 'sim_drawdown'] = 0
#             _df.loc[_df['obs_drawdown'].isna(), 'obs_drawdown'] = 0
#             _df = _df.reset_index(drop = True)
#             return _df


#     df = df.groupby(['site_id']).apply(compute_draw_down)
#     df.to_csv(os.path.join(folder, r"hobs_drawdown.out.csv"))
    
# drawdown_hobs(os.path.join(ml.model_ws, 'Results'),  'hobs.out')

In [ ]:
ml.run_model()

``pyemu`` has a helper function to setup this instruction file for you and also load observations into dataframe

In [ ]:
hob_df = pyemu.gw_utils.modflow_hob_to_instruction_file(
    os.path.join(ml.model_ws,'Results','hobs.out'))

In [ ]:
hob_df

In [ ]:
# tpl_files = [os.path.join(ml.model_ws,f) for f in os.listdir(ml.model_ws) if f.endswith(".tpl")]
# input_files = [f.replace(".tpl",'') for f in tpl_files]
# tpl_files

In [ ]:
tpl_files = ['pest\dataset11a_with_geo.txt.tpl', 
             'pest\dataset11a_with_geo_2015.txt.tpl',
             'pest\dataset11a_with_geo_2016.txt.tpl',
             'pest\pval.PVAL.tpl']
tpl_files = [os.path.join(ml.model_ws,f) for f in tpl_files]
input_files = ['inputs\dataset11a_with_geo.txt',
               'inputs\dataset11a_with_geo_2015.txt',
               'inputs\dataset11a_with_geo_2016.txt',
               'pval.PVAL']
input_files = [os.path.join(ml.model_ws,f) for f in input_files]

In [ ]:
# ins_files = [os.path.join(ml.model_ws,f) for f in os.listdir(ml.model_ws) if f.endswith(".ins")]
# output_files = [f.replace(".ins",'') for f in ins_files]
# ins_files

In [ ]:
ins_files = ['Results\hobs.out.ins']
ins_files = [os.path.join(ml.model_ws,f) for f in ins_files]
output_files =  ['Results\hobs.out']
output_files = [os.path.join(ml.model_ws,f) for f in output_files]

In [ ]:
pst = pyemu.Pst.from_io_files(tpl_files,input_files,ins_files,output_files)

In [ ]:
pst.model_input_data = pst.model_input_data.applymap(lambda x:x.replace("RR_2022\\", ''))
pst.model_output_data = pst.model_output_data.applymap(lambda x:x.replace("RR_2022\\", ''))

In [ ]:
pst.parameter_data.head(10)

In [ ]:
pst.observation_data.head()

Notice the ``obsval`` column?  Let's just set the index of this dataframe to ``obsnme``, then pandas does the hard work for us:

In [ ]:
hob_df.head()

In [ ]:
hob_df.index = hob_df.obsnme
hob_df.head()

In [ ]:
pst.observation_data.loc[hob_df.index,"obsval"] = hob_df.obsval
pst.observation_data.loc[hob_df.index,:].head()

# set observations before may -1 to have weights of 0.5

In [ ]:
def ddown_start_check(xx):
    '''
    make list of observation names that are the first observation from the drawdown series. those should have weights of zero
    '''
    if xx.time_series_data['obsname'][0].decode('UTF-8').startswith('d'):
        return True # if it starts with d that means it's a drawdown, and since we are taking only the first value, it should have a 
    else:
        return False
    
ddown_starts = [obi.time_series_data['obsname'][0].decode('UTF-8')  for obi in obs if ddown_start_check(obi)]

ddown_starts = pd.Series(ddown_starts)

In [ ]:
def weighting(x):
    '''
    set weight to zero if there's no date value on measurement. 
    this value is the first measurement from the drawdown series, 
    meaning it's the raw head. the raw head has already been used
    '''
    v = x.split('.')
    if len(v)>1:
        return  1.0 if  int(v[1])>100 else .5
    else:
        return 1.0
    
hob_df.index.map(weighting)

In [ ]:
pst.observation_data.loc[hob_df.index,"weight"] = hob_df.index.map(weighting)

pst.observation_data.loc[pst.observation_data.obsnme.isin(ddown_starts),'weight'] = 0

display(pst.observation_data.loc[:,"weight"].value_counts().to_frame('value counts'))

pst.observation_data.loc[hob_df.index,:].tail()


In [ ]:
pst.observation_data.obgnme = pst.observation_data.obsnme.apply(lambda x: 'drawdown'  if x.startswith('d') else 'headobs' )

In [ ]:
pst.observation_data.obgnme

BOOM!  that was easy...trying doing that without pandas....not fun!

We still have a few more items to set to specific values. The biggest one is initial values for parameters - they are given default values of 1.0:

In [ ]:
pst.write('RR_2022\pst_temp.pst')

In [ ]:
pst.parameter_data

Luckily, ``pandas`` makes this very easy.  For example, let's set the ``DRN`` conductance parameters to have initial values of mean of the values in the model currently:


In [ ]:
# avg = ml.drn.stress_period_data[0]["cond"].mean()
par = pst.parameter_data #just a pointer to the full, long-named attribute
# drn_pars = par.loc[par.parnme.apply(lambda x: x.startswith("drn")),"parnme"].values
par.loc['swrk',"parval1"] = 2.690747000000000e-006
par.loc['swrk',"parlbnd"] = 1.4e-8
par.loc['swrk',"parubnd"] = 1.4e-4
par.loc['swrk2015',"parval1"] = 2.690747000000000e-006
par.loc['swrk2015',"parlbnd"] = 1.4e-8
par.loc['swrk2015',"parubnd"] = 1.4e-4
par.loc['swrk2016',"parval1"] = 2.690747000000000e-006
par.loc['swrk2016',"parlbnd"] = 1.4e-8
par.loc['swrk2016',"parubnd"] = 1.4e-4
par.loc[par.parnme.str.contains('ss'),'parval1'] = 1e-5
par.loc[par.parnme.str.contains('ss'),'parlbnd'] = 1e-6
par.loc[par.parnme.str.contains('ss'),'parubnd'] = 1e-3
par.loc['ss_3',"parval1"] = 0.1
par.loc['ss_3','parlbnd'] = 1e-5
par.loc['ss_3','parubnd'] = 1e-3
par.loc[par.parnme.str.contains('sy'),'parval1'] = .2
par.loc[par.parnme.str.contains('sy'),'parlbnd'] = 0.05
par.loc[par.parnme.str.contains('sy'),'parubnd'] = 0.3
par.loc[par.parnme.str.contains('vk'),'parval1'] = .25
par.loc[par.parnme.str.contains('vk'),'parlbnd'] = 1e-5
par.loc[par.parnme.str.contains('vk'),'parubnd'] = 1e5
par.loc[par.parnme.str.contains('vk'),'parval1'] = 1.0
par.loc[par.parnme.str.contains('vk'),'parlbnd'] = 1e-5
par.loc[par.parnme.str.contains('vk'),'parubnd'] = 1e5
# #set the par group to mean something
# par.loc[drn_pars,"pargp"] = "drn_cond"
# par.loc[drn_pars,"parubnd"] = avg * 10.0
# par.loc[drn_pars,"parlbnd"] = avg * 0.1
par.loc[:,'pargp'] = par.loc[:,'parnme'].apply(lambda x: x.split("_")[0])

In [ ]:
pst.parrep(r"RR_2022\reg_pest.par", noptmax=50)
par.loc['swrk2015',"parval1"] = 2.690747000000000e-006
par.loc['swrk2016',"parval1"] = 2.690747000000000e-006

par.loc['swrk2015',"scale"] = 1.0
par.loc['swrk2016',"scale"] = 1.0
par.loc['swrk2015',"offset"] =0.0
par.loc['swrk2016',"offset"] = 0.0

assert par.isnull().sum().sum()==0, 'fail'

In [ ]:
pst.model_command

In [ ]:
pst.model_command = [r"python pest\forward_run.py"]

Let's save this version of the control file

In [ ]:
pst = pyemu.pst.lo

In [ ]:
pst.write(os.path.join(ml.model_ws,"pest.pst"))

### adding prior information

``pyemu`` supports both zero-order (preferred value) and first-order (preferred difference) Tikhonov regularization.  Let's set preferred value for the conductance parameters:

In [ ]:
pyemu.utils.helpers.zero_order_tikhonov(pst)
pst.prior_information.head()

In [ ]:
pst.control_data.pestmode = "regularization"

### setting PEST++ options
Some things I like to add:

In [ ]:
# pst.write(os.path.join(ml.model_ws,'pest',"reg_pest.pst"))
pst.write(os.path.join(ml.model_ws,"reg_pest.pst"))

# run this after completion to replace with best fit parms

In [ ]:
# before continuing, we are going to re-load the pest control file to get rid of any changes we introduced
pst = pyemu.Pst(os.path.join(ml.model_ws,"pp_pest.pst"))

In [ ]:
# pst = pyemu.Pst(r"RR_2022\reg_pest.pst")
pst.parrep(r"RR_2022\pp_pest.par")
# pst.write(os.path.join('RR_2022',"pp_pest_2.pst"))
# pst.write(my_new.pst")
# #write a version 2 control file
# pst.write("my_new_v2.pst",version=2)

In [ ]:
pst.write(os.path.join('RR_2022',"pp_pest_2.pst"))

In [ ]:
pst.control_data.noptmax = 50

In [ ]:
pst.write(os.path.join(ml.model_ws,"reg_pest.pst"))

In [ ]:
# os.path.exists(os.path.join(ml.model_ws,'reg_pest.par'))

In [ ]:
for year in range(2012,2021):
    run = f"June{year}"
    m = basic.load_model(path = os.path.join('temp', run))
    p = os.path.join('initial_heads', run)
    initial_conditions.set_starting_heads(m, plot = False, alt_outpath=p)

In [ ]:
pst = pyemu.gw_utils.load

In [ ]:
pst.parrep(os.path.join(ml.model_ws,'reg_pest.par'))

In [ ]:
def load_rei(m, file  = 'reg_pest.rei'):
    rei = pd.read_csv(os.path.join(m.model_ws, file), skiprows= [0,1,2], delim_whitespace=True)
    rei = rei.query("Group.str.contains('ob')")

    rei.loc[:,'date'] = pd.to_datetime('1/1/2014') + pd.to_timedelta(
        rei.loc[:,'Name'].apply(lambda x: int(x.split('.')[1])), unit = "D")

    rei.loc[:,'Well'] = rei.loc[:,'Name'].apply(lambda x: x.split('.')[0])
    rei.loc[:,'phi'] = rei.Weight*rei.Residual**2
    
    return rei

def plot_residuals(rei):
    for well, df in rei.groupby('Well'):
        # equivalent but more general
        plt.figure( figsize = (10,10))
        ax = plt.subplot(2, 1, 1)
        ax.grid(True)
        ax.set_title(well)
        # add a subplot with no frame
        # ax2 = plt.subplot(222, frameon=False)

        ax.scatter(df.date, df.Measured, label = 'Measured')
        ax.scatter(df.date, df.Modelled, label = 'Modelled')

        ax2 = plt.subplot(212, frameon=True)
        ax2.grid(True)
        ax2.scatter(df.date, df.phi, label = 'Residual', color = 'r')
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax2.text(1,0, f"perc {100*df.phi.sum()/rei.phi.sum():.1f}%\nphi = {df.phi.sum():.0f}",
                 fontsize=14,
            verticalalignment='bottom', bbox=props,
                 transform = ax.transAxes)
        ax.legend()
        ax2.legend(bbox_to_anchor = (1,1), loc = 'upper left')

In [ ]:
rei = load_rei(ml , 'pp_pest.rei')

In [ ]:
pd.to_datetime('1/1/2012')-pd.to_datetime('5/1/2012')

In [ ]:
plot_residuals(rei)
    